## Importing the libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

## Checking the data

In [ ]:
df = pd.read_csv('../input/housesalesprediction/kc_house_data.csv')

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.describe().transpose()

## Exploratory Data Analysis

In [ ]:
plt.figure(figsize = (12,8))
sns.distplot(df['price'])

In [ ]:
plt.figure(figsize = (10,6))
sns.countplot(df['bedrooms'])

In [ ]:
hist2 = [go.Histogram(x=df.yr_built,xbins=dict(start=np.min(df.yr_built),size=1,end=np.max(df.yr_built)),marker=dict(color='rgb(0,102,0)'))]

histlayout2 = go.Layout(title="Built Year Counts",xaxis=dict(title="Years"),yaxis=dict(title="Built Counts"))

histfig2 = go.Figure(data=hist2,layout=histlayout2)

iplot(histfig2)

In [ ]:
sns.countplot(x='floors',data=df, palette='Set2')

In [ ]:
plt.figure(figsize = (12,8))
sns.scatterplot(x='price',y='sqft_living',data=df)

In [ ]:
plt.figure(figsize = (12,8))
sns.boxplot(x='bedrooms',y='price',data=df)

In [ ]:
sns.boxplot(x='waterfront',y='price',data=df)

## Geographical Plotting

In [ ]:
plt.figure(figsize = (12,8))
sns.scatterplot(x='price',y='long',data=df)

In [ ]:
plt.figure(figsize = (12,8))
sns.scatterplot(x='price',y='lat',data=df)

In [ ]:
plt.figure(figsize = (12,8))
sns.scatterplot(x='long',y='lat',data=df,hue='price')

In [ ]:
df.sort_values('price',ascending=False).head(20)

In [ ]:
len(df)*0.01

In [ ]:
non_top_1_perc = df.sort_values('price',ascending=False).iloc[216:]

In [ ]:
plt.figure(figsize = (12,8))
sns.scatterplot(x='long',y='lat',data=non_top_1_perc,hue='price',palette='RdYlGn',edgecolor=None,alpha=0.2)

## Working with feature data

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df = df.drop('id',axis=1)

In [ ]:
df.head()

## Feature engineering from date

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df['month'] = df['date'].apply(lambda date:date.month)

In [ ]:
df['year'] = df['date'].apply(lambda date:date.year)

In [ ]:
df.head(2)

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(x='year',y='price',data=df)

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(x='month',y='price',data=df)

In [ ]:
df.groupby('month').mean()['price'].plot()

In [ ]:
df = df.drop('date',axis=1)

In [ ]:
df = df.drop('zipcode',axis=1)

In [ ]:
df.head()

In [ ]:
X = df.drop('price',axis=1)
y = df['price']

## Train-test Split 

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

## Scaling the data

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
X_train = scaler.fit_transform(X_train)

In [ ]:
X_test = scaler.transform(X_test)

In [ ]:
X_train.shape

## Creating a model

In [ ]:
from tensorflow.keras.models import Sequential

In [ ]:
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential()

model.add(Dense(19,activation='relu'))
model.add(Dense(19,activation='relu'))
model.add(Dense(19,activation='relu'))
model.add(Dense(19,activation='relu'))

model.add(Dense(1))

model.compile(optimizer='adam',loss='mse')

## Training the  model

In [ ]:
model.fit(x=X_train,y=y_train.values,
          validation_data=(X_test,y_test.values),
          batch_size=128,epochs=400)

In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
losses.plot()

## Evaluation on test data 

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,explained_variance_score

In [ ]:
predictions = model.predict(X_test)

In [ ]:
mean_absolute_error(y_test,predictions)

In [ ]:
mean_squared_error(y_test,predictions)

In [ ]:
mean_squared_error(y_test,predictions)**0.5

In [ ]:
explained_variance_score(y_test,predictions)

In [ ]:
# Our predictions
plt.figure(figsize=(10,6))
plt.scatter(y_test,predictions)

# Perfect predictions
plt.plot(y_test,y_test,'r')

## Comparing with linear regression

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(X_train,y_train)

In [ ]:
predictionslr = lr.predict(X_test)

In [ ]:
mean_absolute_error(y_test,predictionslr)

In [ ]:
mean_squared_error(y_test,predictionslr)

In [ ]:
mean_squared_error(y_test,predictionslr)**0.5

In [ ]:
explained_variance_score(y_test,predictionslr)